<a href="https://colab.research.google.com/github/AryanAhmadChaudhary/Hybrid-ADR-Detection-Rule-Mining-Machine-Learning/blob/main/ADR_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DEPENDENCIES


In [25]:
import nltk
nltk.download('punkt_tab')
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [26]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from datasets import load_dataset
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from mlxtend.frequent_patterns import apriori, association_rules

# LOAD DATASET

In [27]:
from datasets import load_dataset

dataset = load_dataset("SetFit/ade_corpus_v2_classification")

train_df = pd.DataFrame(dataset["train"])
test_df = pd.DataFrame(dataset["test"])

df = pd.concat([train_df, test_df]).reset_index(drop=True)

df.drop("label_text", axis=1, inplace=True)

df.head()


Repo card metadata block was not found. Setting CardData to empty.


,text,label
0,"On cessation of the injections, the retrocorne...",0
1,Median patient age was 52 years.,0
2,A whole brain irradiation was performed for 37...,0
3,Complex biochemical syndrome of hypocalcemia a...,0
4,The fastidious organism grew only on buffered ...,0


# TEXT CLEANING

In [28]:
stop_words = set(stopwords.words("english"))

def clean_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [t for t in tokens if t.isalpha() and t not in stop_words]
    return " ".join(tokens)

df["clean_text"] = df["text"].apply(clean_text)


# RULE-BASED MODEL (Apriori)

In [29]:
from sklearn.feature_extraction.text import CountVectorizer
from mlxtend.frequent_patterns import apriori, association_rules

vectorizer_rules = CountVectorizer(max_features=300, binary=True)
word_matrix = vectorizer_rules.fit_transform(df["clean_text"]).toarray()

words_df = pd.DataFrame(word_matrix, columns=vectorizer_rules.get_feature_names_out())
words_df["ADR"] = df["label"]

frequent = apriori(words_df, min_support=0.01, use_colnames=True)

rules = association_rules(frequent, metric="lift", min_threshold=1.1)


rule_triggers = {}
for _, row in rules.iterrows():
    if row["consequents"] == frozenset({"ADR"}) and len(row["antecedents"]) == 1:
        word = list(row["antecedents"])[0]
        conf = row["confidence"]
        rule_triggers[word] = conf




## Rule-Based Predictor (returns prediction + confidence)

In [30]:
def rule_based_predict_with_conf(text):
    tokens = clean_text(text).split()
    best_conf = 0

    for word in tokens:
        if word in rule_triggers:
            conf = rule_triggers[word]
            if conf > best_conf:
                best_conf = conf

    if best_conf > 0:
        return 1, best_conf
    return 0, 0


# TRADITIONAL ML MODELS

## Data Split

In [31]:
from sklearn.model_selection import train_test_split

vectorizer = CountVectorizer(max_features=2000)
X = vectorizer.fit_transform(df["clean_text"])
y = df["label"]

X_train, X_test, y_train, y_test, text_train, text_test = train_test_split(
    X, y, df["text"], test_size=0.2, random_state=42
)


## Models Trainig

### Logistic Regression

In [32]:
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(max_iter=300)
lr_model.fit(X_train, y_train)


LogisticRegression(max_iter=300)

### Support vector Machine (SVM)

In [33]:
from sklearn.svm import LinearSVC

svm_model = LinearSVC()
svm_model.fit(X_train, y_train)


LinearSVC()

## Models Predictions

### Logistic regression

In [34]:
lr_pred = lr_model.predict(X_test)

### SVM

In [35]:
svm_pred = svm_model.predict(X_test)

# HYBRID MODELS (Rule + ML)

##Hybrid Logistic Regression

In [36]:
def hybrid_lr_predict(text):
    rule_pred, rule_conf = rule_based_predict_with_conf(text)
    ml_pred = lr_model.predict(vectorizer.transform([text]))[0]

    if rule_pred == 1 and rule_conf >= 0.80:
        return 1
    return ml_pred



##Hybrid SVM

In [37]:
def hybrid_svm_predict(text):
    rule_pred, rule_conf = rule_based_predict_with_conf(text)
    ml_pred = svm_model.predict(vectorizer.transform([text]))[0]

    if rule_pred == 1 and rule_conf >= 0.80:
        return 1
    return ml_pred


## Hybrid predictions

In [38]:
hybrid_lr_preds  = [hybrid_lr_predict(t)  for t in text_test]
hybrid_svm_preds = [hybrid_svm_predict(t) for t in text_test]


# EVALUATION

In [39]:
from sklearn.metrics import accuracy_score, f1_score

def evaluate(name, y_true, y_pred):
    return {
        "Model": name,
        "Accuracy": accuracy_score(y_true, y_pred),
        "F1 Score": f1_score(y_true, y_pred)
    }

results = []


## Rule-Based

In [40]:

rule_preds = [rule_based_predict_with_conf(t)[0] for t in text_test]
results.append(evaluate("Rule-Based", y_test, rule_preds))


## ML Models

### Logistic Regression

In [41]:
results.append(evaluate("Logistic Regression", y_test, lr_pred))


### SVM

In [42]:
results.append(evaluate("SVM", y_test, svm_pred))


## Hybrid Models

### Hybrid Logistic Regression

In [43]:
results.append(evaluate("Hybrid (Rule + LR)", y_test, hybrid_lr_preds))


### Hybrid SVM

In [44]:
results.append(evaluate("Hybrid (Rule + SVM)", y_test, hybrid_svm_preds))


# FINAL COMPARISON TABLE

In [45]:
df_results = pd.DataFrame(results)
df_results


,Model,Accuracy,F1 Score
0,Rule-Based,0.512968,0.498797
1,Logistic Regression,0.854167,0.724055
2,SVM,0.846726,0.721514
3,Hybrid (Rule + LR),0.867985,0.760694
4,Hybrid (Rule + SVM),0.859269,0.752801
